In [1]:
from datetime import datetime
import pandas_datareader as reader

df = reader.get_data_yahoo(['BNB-USD'], start=datetime(2020,11,11))
df


Attributes   Adj Close       Close        High         Low        Open  \
Symbols        BNB-USD     BNB-USD     BNB-USD     BNB-USD     BNB-USD   
Date                                                                     
2020-11-11   27.956179   27.956179   28.620056   27.871515   28.193321   
2020-11-12   27.614807   27.614807   28.157188   27.349792   27.955782   
2020-11-13   28.182173   28.182173   28.481514   27.562483   27.614275   
2020-11-14   27.936201   27.936201   28.182173   27.409859   28.182173   
2020-11-15   27.621031   27.621031   28.172455   27.114182   27.936201   
...                ...         ...         ...         ...         ...   
2021-11-06  636.113953  636.113953  636.113953  596.764465  608.263489   
2021-11-07  650.454041  650.454041  669.345825  632.079285  638.186035   
2021-11-08  654.315002  654.315002  655.713684  632.687073  650.467407   
2021-11-09  635.190613  635.190613  657.040588  632.331665  654.000366   
2021-11-10  596.580750  596.580750  661.009216  587.544128  633.338989   

Attributes      Volume  
Symbols        BNB-USD  
Date                    
2020-11-11   279645463  
2020-11-12   266092376  
2020-11-13   292737428  
2020-11-14   254114574  
2020-11-15   254946427  
...                ...  
2021-11-06  2779927044  
2021-11-07  3017167868  
2021-11-08  2828112534  
2021-11-09  2198989754  
2021-11-10  3398239744  

[365 rows x 6 columns]

In [2]:
import pandas as pd

pd.plotting.register_matplotlib_converters()

print(df.info())

df['Close'].plot(figsize=(16,6))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 365 entries, 2020-11-11 to 2021-11-10
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   (Adj Close, BNB-USD)  365 non-null    float64
 1   (Close, BNB-USD)      365 non-null    float64
 2   (High, BNB-USD)       365 non-null    float64
 3   (Low, BNB-USD)        365 non-null    float64
 4   (Open, BNB-USD)       365 non-null    float64
 5   (Volume, BNB-USD)     365 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 20.0 KB
None


In [3]:
df[df.index.duplicated()]

Attributes   Adj Close       Close        High         Low       Open  \
Symbols        BNB-USD     BNB-USD     BNB-USD     BNB-USD    BNB-USD   
Date                                                                    
2021-03-28  269.191284  269.191284  280.992401  262.288666  269.31073   

Attributes      Volume  
Symbols        BNB-USD  
Date                    
2021-03-28  2527142069

In [4]:
df = df[~df.index.duplicated(keep='first')]
df = df['Close']

In [5]:
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()
datastore = ws.get_default_datastore()

df_sans_last_60_days = df[:-30]
df_sans_last_60_days.reset_index(inplace=True)
training_data = Dataset.Tabular.register_pandas_dataframe(
    df_sans_last_60_days, datastore, 'BNBRate')

Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/data/dataset_factory.py:1216: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = pd.to_datetime(df[column])


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/a8237fef-cc0a-40c0-8c8a-7448fe704194/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute

compute_name = 'shrek'

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print(f'Using existing compute: {compute_name}')    
else:
    print(f'Creating new compute: {compute_name}')
    provisioning_config = AmlCompute.provisioning_configuration(
        vm_size = 'Standard_F4s_v2',
        min_nodes = 0, 
        max_nodes = 4)

    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    compute_target.wait_for_completion(show_output=True)

Creating new compute: shrek
InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
from azureml.train.automl import AutoMLConfig
from azureml.automl.core.forecasting_parameters import ForecastingParameters

forecasting_parameters = ForecastingParameters(
    time_column_name='Date',
    freq='1D',
    forecast_horizon=1,
    feature_lags='auto',
    target_lags=[7],
    use_stl=None,
    country_or_region_for_holidays='US',
)

In [9]:
automl_config = AutoMLConfig(
    task = "forecasting",
    primary_metric='normalized_root_mean_squared_error',

    training_data = training_data,
    label_column_name = 'BTC-USD',
    n_cross_validations=7,

    enable_early_stopping=True,
    early_stopping_n_iters=20,
    max_concurrent_iterations=4,
    iteration_timeout_minutes=5,

    compute_target=compute_target,
    forecasting_parameters=forecasting_parameters
)

In [10]:
from azureml.core import Experiment
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()
experiment = Experiment(ws, "AutoML-Forecasting")
run = experiment.submit(automl_config, show_output=True)


best_run, best_model = run.get_output()

Submitting remote run.
No run_configuration provided, running on shrek with default configuration
Running on remote compute: shrek


In [33]:
from azureml.core import Experiment
from azureml.train.automl.run import AutoMLRun

ws = Workspace.from_config()
datastore = ws.get_default_datastore()

experiment = Experiment(ws, "AutoML-Forecasting")
runs = list(experiment.get_runs()) 

run = AutoMLRun(experiment, runs[0].id)
best_run, best_model = run.get_output()

Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


Performing interactive authentication. Please follow the instructions on the terminal.


KeyboardInterrupt: 

In [ ]:
from datetime import datetime

range = pd.date_range(datetime(2021,10,10), datetime(2021,10,20))

forecast_df = pd.DataFrame({ 'Date': range })

In [ ]:
forecast, transformed_df = best_model.forecast(forecast_df)

In [ ]:
pretty_df = forecast_df.copy().set_index('Date')
pretty_df['ETH-USD-forecast'] = forecast
pretty_df['ETH-USD'] = df['ETH-USD']

pretty_df[['ETH-USD', 'ETH-USD-forecast']].plot(figsize=(16,6))